In [1]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import time

# URL de la page d'accueil de Hespress
base_url = "https://www.hespress.com/all"

# Connexion à MongoDB
client = MongoClient("mongodb://localhost:27017/")  # Remplacez par votre chaîne de connexion MongoDB
db = client["hespressArabic_db"]  # Nom de la base de données
collection = db["articles"]  # Nom de la collection

# En-têtes HTTP pour simuler une requête de navigateur
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Catégories à scraper
CATEGORIES = ["سياسة", "اقتصاد", "رياضة", "مجتمع ", "فن وثقافة"]

# Fonction pour vérifier si un article appartient à une catégorie spécifique
def is_article_in_category(article, category):
    category_element = article.find('span', class_='cat')
    if category_element and category_element.text.strip().upper() == category.upper():
        return True
    return False

# Fonction pour extraire le contenu d'un article
def extract_article_content(article_url):
    try:
        response = requests.get(article_url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            # Sélecteur pour le contenu de l'article (à adapter selon la structure du site)
            content_element = soup.find('div', class_='article-content')
            if content_element:
                return content_element.text.strip()
        return "No Content"
    except Exception as e:
        print(f"Erreur lors de l'extraction du contenu de {article_url} : {e}")
        return "No Content"

# Fonction pour scraper les articles
def scrape_articles():
    page_number = 1
    while collection.count_documents({}) < 5000:  # Continuer jusqu'à 5000 articles
        # Construire l'URL de la page (si le site utilise une pagination)
        page_url = f"{base_url}/page/{page_number}"  # Exemple de pagination
        print(f"Scraping page {page_number} : {page_url}")

        # Récupérer le contenu de la page
        response = requests.get(page_url, headers=headers)
        if response.status_code != 200:
            print(f"Erreur : Impossible d'accéder à la page {page_number}")
            break

        # Parser le contenu HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Trouver tous les articles sur la page
        articles = soup.find_all('div', class_='overlay card')  # Sélecteur pour les articles
        if not articles:
            print("Aucun article trouvé sur cette page.")
            break

        for article in articles:
            # Extraire le titre
            title_element = article.find('h3', class_='card-title')
            title = title_element.text.strip() if title_element else 'No Title'

            # Extraire la date de publication
            date_element = article.find('small', class_='text-muted time')
            date = date_element.text.strip() if date_element else 'No Date'

            # Extraire la catégorie
            category_element = article.find('span', class_='cat')
            category = category_element.text.strip() if category_element else 'No Category'

            # Extraire le lien de l'article
            link_element = article.find('a', class_='stretched-link')
            link = link_element['href'] if link_element else None

            # Filtrer par catégorie
            if category.upper() in CATEGORIES and link:
                # Extraire le contenu de l'article
                content = extract_article_content(link)

                # Créer un document JSON pour MongoDB
                article_data = {
                    'title': title,
                    'date': date,
                    'category': category,
                    'content': content
                }

                # Insérer le document dans MongoDB
                collection.insert_one(article_data)
                print(f"Article ajouté : {title} (Catégorie : {category}, Date : {date})")

                # Si nous avons atteint 5000 articles, sortir de la boucle
                if collection.count_documents({}) >= 5000:
                    break

        # Passer à la page suivante
        page_number += 1
        time.sleep(2)  # Délai pour éviter de surcharger le serveur

# Appeler la fonction pour scraper les articles
try:
    scrape_articles()
    print("Scraping terminé. Les données ont été sauvegardées dans MongoDB.")
except Exception as e:
    print(f"Erreur lors du scraping : {e}")

Scraping page 1 : https://www.hespress.com/all/page/1
Article ajouté : المغرب وفرنسا يعززان الشراكة الثقافية بتوقيع اتفاقيات جديدة بالرباط (Catégorie : سياسة, Date : الثلاثاء 18 فبراير 2025 - 14:32)
Article ajouté : مهنيو التمور بجهة درعة تافيلالت يحذرون من المنتجات الجزائرية المهربة (Catégorie : اقتصاد, Date : الثلاثاء 18 فبراير 2025 - 14:00)
Article ajouté : رحو: الأسعار تتضاعف بالمضاربة .. وتقييم استيراد اللحوم مسؤولية الحكومة (Catégorie : اقتصاد, Date : الثلاثاء 18 فبراير 2025 - 13:00)
Article ajouté : الملك محمد السادس يهنئ غامبيا (Catégorie : سياسة, Date : الثلاثاء 18 فبراير 2025 - 12:56)
Scraping page 2 : https://www.hespress.com/all/page/2
Article ajouté : ولعلو يشرح تصميم "ملعب بنسليمان" (Catégorie : رياضة, Date : الثلاثاء 18 فبراير 2025 - 12:31)
Article ajouté : بوطازوت ترفض الاستفادة من راحة (Catégorie : فن وثقافة, Date : الثلاثاء 18 فبراير 2025 - 11:44)
Scraping page 3 : https://www.hespress.com/all/page/3
Article ajouté : تصريحات "متعددي مصادر الدخل" تحت مجهر مصالح المراقب